### This notebook contains code to extract 30 seconds raw audio using spotify api. 

In [1]:
#importing module spotipy to make api calls to spotify
import spotipy
# module to extract song from url
import urllib
# OAuth Credenials
from spotipy.oauth2 import SpotifyClientCredentials
from multiprocessing import Process, Queue
import pandas as pd 

### Authentication

In [22]:
client_credentials_manager = SpotifyClientCredentials(client_id='4a1a457d14c0445a96e7021a5523905b', client_secret='45b186d601544fdcbb4fe07c7b23cce5')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


In [75]:
import time

class Timer(object):
    def __init__(self, verbose=True):
        self.verbose = verbose

    def __enter__(self):
        self.start = time.time()
        return self

    def __exit__(self, *args):
        self.end = time.time()
        self.secs = self.end - self.start
        self.msecs = self.secs * 1000  # millisecs
        if self.verbose:
            print('elapsed time: %f ms %f projected finish' % (self.msecs, self.msecs*1726/(4*1000*3600)))

In [24]:
def get_track_data(offset, numLoops, queue, limit=50):
    # store all tracks of all playlists 
    print('Starting')
    track_ids = []
    track_names =[]
    preview_urls_all = []
    track_popularity = []
    for loopIndex in range(numLoops):
        playlists = sp.user_playlists('spotify', limit=limit, offset=(offset + limit*loopIndex))
        
        for i, playlist in enumerate(playlists['items']):
            try:
                with Timer() as t:
                    metadata = sp.user_playlist('spotify', playlist_id=playlist['id'],
                        fields='tracks.items(track.name, track.id)')
                    
                    
                    for index, item in enumerate(metadata['tracks']['items']):
                        track_ids.append(item['track']['id'])
                        track_names.append(item['track']['id'])

                        track_popularity.append(item['track']['id'])
#                     track_details = sp.tracks(tracks=all_ids)
#                     for track in track_details['tracks']:
#                         preview_urls_all.append(track['preview_url'])

#                         urllib.urlretrieve (track['preview_url'], "track"+str(track['id'])+".wav")
#                         print('track converted')

                    print("%4d %s" % (i + 1 + playlists['offset'], playlist['name']))
            except:
            #    a=0
                print("NO METADATA")
        if playlists['next']==None:
            break

    track_data = pd.DataFrame({
    'track_id': track_ids,
    'track_name': track_names,
    
   
    })
    track_data.to_csv("track_data" + str(track_ids[0])+ ".csv")

    q.put(track_data)
    name = multiprocessing.current_process().name
    print(name, 'Exiting')


In [7]:
if __name__ == '__main__':
    q = Queue()
    for offset in [0,400,800,1200,1600]:
        Process(target=get_track_data, args=(offset,8,q)).start()
    #p.start()
    #print(q.get())    # prints "[42, None, 'hello']"
    #p.join()

Starting
Starting
Starting
Starting
Starting
   1 Today's Top Hits
elapsed time: 321.603060 ms 0.038548 projected finish
 401 Vintage Vibes
elapsed time: 437.333107 ms 0.052419 projected finish
1201 Trip-Hop Station
1601 Music for a Workday
elapsed time: 397.699118 ms 0.047669 projected finish
elapsed time: 344.945908 ms 0.041346 projected finish
   2 Rap Caviar
elapsed time: 1008.655071 ms 0.120899 projected finish
 402 Indie Bluegrass
elapsed time: 1244.023085 ms 0.149110 projected finish
1202 The Britney Body Workout
elapsed time: 1224.921942 ms 0.146821 projected finish
1602 Fest!
elapsed time: 1232.187986 ms 0.147691 projected finish
   3 electroNOW
elapsed time: 488.250971 ms 0.058522 projected finish
1203 Carefree Days
elapsed time: 269.604921 ms 0.032315 projected finish
1603 Pepplåtar till träningspasset
elapsed time: 369.786024 ms 0.044323 projected finish
 403 Fresh Folk | 2017
elapsed time: 736.537218 ms 0.088282 projected finish
1204 Confidence Boost
elapsed time: 553.3740

In [ ]:
all_dfs = []
for _ in range(5):
    all_dfs.append(q.get())

track_dataframe = all_dfs[0]
for df in all_dfs[1:]:
    track_dataframe = track_dataframe.append(df,ignore_index=True)

In [71]:
# find the no. of unique songs in dataframe
unique_tracks = track_dataframe.track_id.unique()

print("No. of unique songs :", len(unique_tracks))

('No. of unique songs :', 60110)


In [79]:
# code ro extract url for raw-audio and then download audio in wav format
def get_preview_urls(offset, numLoops, queue, track_ids, limit=10):
    # store all tracks of all playlists 
    print('Starting')
    track_ids_all= []
    preview_urls_all = []

    for loopIndex in range(numLoops):
        start = offset + limit*loopIndex
        end = start +50
        tracks_list = sp.tracks(track_ids[start:end])
        
        
        for track in tracks_list['tracks']:
            try:
                with Timer() as t:
                    
                    preview_urls_all.append(track['preview_url'])
                    track_ids_all.append(track['id'])
                    urllib.urlretrieve(track['preview_url'], "tracks_total/track"+track['id']+".wav")
            except:
            #    a=0
                print("NO METADATA")
        

    track_data = pd.DataFrame({
    'preview_url': preview_urls_all,
    'id': track_ids_all,
    
   
    })
    track_data.to_csv("preview_urls/preview_url" + str(track_ids[0])+ ".csv")


    #name = multiprocessing.current_process().name
    print('Exiting')


In [ ]:
# since there are 60000 songs, so this process needs to be repeated for 60000 songs, I have skipped adding code 
# for that since it is repetitive except the offset values.
if __name__ == '__main__':
    q = Queue()
    for offset in [0,500,1000,1500]:
        Process(target=get_preview_urls, args=(offset,10,q,unique_tracks)).start()


Extraction of 34 acoustic features from audio samples at the rate of 34 samples per second.

In [ ]:
The following code needs to be run in terminal